<a href="https://colab.research.google.com/github/Whoseyashar/Machine-Learning-Advance/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One of the most popular applications of GNNs is in Recommendation Systems. If you think about the foundation of Word2Vec (and, thus, DeepWalk and Node2Vec), the goal is to produce vectors with the ability to measure their similarity. Encode movies instead of words, and you can suddenly ask for movies that are the most similar to a given input title.

But how to encode movies? We want to create (biased) random walks of movies, but this requires a graph dataset where similar movies are connected to each other and this is not easy to find. Yet, we can build our own. One approach is to take a dataset of movies, and look for user ratings.

There are different techniques to build a graph based on ratings: bipartite graphs, edges based on pointwise mutual information, and so on. We will implement a simple and intuitive approach: movies that are liked by the same users are connected.

First lets download a dataset comprising of 100,836 ratings, 9,742 movies, and 610 users.

In [1]:
import os
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

with urlopen(url) as zurl:
    with ZipFile(BytesIO(zurl.read())) as zfile:
        zfile.extractall('.')

We are interested in two files: ratings.csv and movies.csv. The first one stores all the ratings made by users, and the second one allows us to translate movie identifiers into titles. Let’s see what they look like by importing them with `pandas`

In [2]:
import pandas as pd
import networkx as nx

ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'unix_timestamp'])

ratings


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [3]:
movies = pd.read_csv('ml-100k/u.item', sep='|', usecols=range(2), names=['movie_id', 'title'], encoding='latin-1')

movies

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


We want to see movies that have been liked by the same users. This means that ratings such as 1, 2, and 3 are not very relevant. We can discard those and only keep scores of 4 and 5.

In [4]:
ratings = ratings[ratings.rating >= 4]
ratings

,user_id,movie_id,rating,unix_timestamp
5,298,474,4,884182806
7,253,465,5,891628467
11,286,1014,5,879781125
12,200,222,5,876042340
16,122,387,5,879270459
...,...,...,...,...
99988,421,498,4,892241344
99989,495,1091,4,888637503
99990,806,421,4,882388897
99991,676,538,4,892685437


The next step is to count every time that two movies are liked by the same user. We will repeat this process for every user in the dataset.

In [5]:
from collections import defaultdict

pairs = defaultdict(int)

for group in ratings.groupby("user_id"):
    user_movies = list(group[1]["movie_id"])
    for i in range(len(user_movies)):
        for j in range(i + 1, len(user_movies)):
            pairs[(user_movies[i], user_movies[j])] += 1

G = nx.Graph()
for pair in pairs:
    movie1, movie2 = pair
    score = pairs[pair]

    if score >= 20: # We don’t consider lower scores because that would create a large graph in which connections were less meaningful
        G.add_edge(movie1, movie2, weight=score)

print(G)

Graph with 410 nodes and 14936 edges


The graph we created has 410 nodes (movies) and 14,936 edges. We can now train Node2Vec on it to learn the node embeddings!

We could reuse our implementation from the previous section, but there is actually an entire Python library dedicated to Node2Vec. (note: the training will take a couple of minutes, get used to it..)

In [6]:
!pip install node2vec
from node2vec import Node2Vec

node2vec = Node2Vec(G, dimensions=64, walk_length=20, num_walks=200, p=2, q=1, workers=1)

model = node2vec.fit(window=10, min_count=1, batch_words=4)

Computing transition probabilities:   0%|          | 0/410 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 200/200 [00:37<00:00,  5.39it/s]


The Node2Vec model is trained and we can now use it the same way we use the Word2Vec object from the `gensim` library.

Now let's create a function that will recommend movies based on a given title. It starts by converting the title into a movie ID we can use to query our model. We loop through the five most similar word vectors. We convert these IDs into movie titles that we print with their corresponding similarity scores.

In [7]:
def recommend(movie):
    movie_id = str(movies[movies.title == movie].movie_id.values[0])
    for id in model.wv.most_similar(movie_id)[:5]:
        title = movies[movies.movie_id == int(id[0])].title.values[0]
        print(f'{title}: {id[1]:.2f}')


recommend('Star Wars (1977)')

Return of the Jedi (1983): 0.58
Raiders of the Lost Ark (1981): 0.55
Toy Story (1995): 0.51
Indiana Jones and the Last Crusade (1989): 0.49
Silence of the Lambs, The (1991): 0.48


In [8]:
movie = 'Star Wars (1977)'
movie_id = str(movies[movies.title == movie].movie_id.values[0])

Exercise:
1.  play with different hyperparameters of the model and check for improvement of the recommendations.
2.  try DeepWalk and compare the accuracy

In [9]:
import numpy as np

In [10]:
def random_walk(start, length):
    walk = [str(start)]  # starting node
    for i in range(length):
        neighbors = [node for node in G.neighbors(start)]
        next_node = np.random.choice(neighbors, 1)[0]
        walk.append(str(next_node))
        start = next_node
    return walk


In [11]:
walks = []
for node in G.nodes:
    for _ in range(80):
        walks.append(random_walk(node, 10))

In [12]:
from gensim.models import Word2Vec

model = Word2Vec(walks,
                 hs=1,  # Hierarchical softmax
                 sg=1,  # Skip-gram
                 vector_size=100,
                 window=10,
                 workers=2,
                 seed=0)

In [13]:
model.train(walks, total_examples=model.corpus_count, epochs=30, report_delay=1)

(7811023, 10824000)

In [14]:
print('Nodes that are the most similar to node 0:')
for similarity in model.wv.most_similar(positive=[movie_id]):
    print(f'   {similarity}')
    print(f'   {movies[movies.movie_id == int(similarity[0])].title.values[0]}  : {similarity[1]}')

Nodes that are the most similar to node 0:
   ('1028', 0.5360468626022339)
   Grumpier Old Men (1995)  : 0.5360468626022339
   ('100', 0.5125670433044434)
   Fargo (1996)  : 0.5125670433044434
   ('174', 0.49770599603652954)
   Raiders of the Lost Ark (1981)  : 0.49770599603652954
   ('181', 0.4712142050266266)
   Return of the Jedi (1983)  : 0.4712142050266266
   ('98', 0.457556813955307)
   Silence of the Lambs, The (1991)  : 0.457556813955307
   ('277', 0.45182329416275024)
   Restoration (1995)  : 0.45182329416275024
   ('1', 0.3869374990463257)
   Toy Story (1995)  : 0.3869374990463257
   ('924', 0.3857589066028595)
   White Squall (1996)  : 0.3857589066028595
   ('172', 0.38382065296173096)
   Empire Strikes Back, The (1980)  : 0.38382065296173096
   ('127', 0.37875857949256897)
   Godfather, The (1972)  : 0.37875857949256897
